# 07 — Results & Discussion

## Objectif

Analyser et interpréter les résultats obtenus dans :

- Graph Analysis (structure)
- Community Detection
- Link Prediction
- Classification

Nous discutons :
- Des propriétés structurelles du graphe
- Du rôle des communautés
- De la comparaison structure vs sémantique
- De l’impact des features sur la classification
- Des limites du projet
- Des perspectives futures

In [1]:
import os
import json
import pandas as pd
import numpy as np

## 1) Analyse structurelle du graphe

On charge les métriques calculées dans le Notebook 03.

In [2]:
with open("results/graph_metrics.json", "r") as f:
    graph_metrics = json.load(f)

graph_metrics

{'n_nodes': 390,
 'n_edges': 1200,
 'density': 0.015819655922483685,
 'n_components': 1,
 'largest_component': 390,
 'avg_degree_cv': 4.0,
 'avg_degree_job': 13.333333333333334,
 'max_degree_cv': 4,
 'max_degree_job': 29}

### Interprétation détaillée des propriétés structurelles

#### 1️⃣ Taille du graphe

Le graphe contient :

- **390 nœuds**
- **1200 arêtes**

Ce volume correspond à un marché simulé de taille moyenne,
avec un nombre raisonnable d’interactions CV–Job.

---

#### 2️⃣ Densité du graphe (≈ 0.0158)

La densité est faible (~1.6%).

Cela signifie que :

- Seule une petite fraction des paires CV–Job possibles sont connectées.
- Le graphe est **clairsemé**, ce qui est cohérent avec un système réel de recrutement :
  un CV ne correspond qu'à un sous-ensemble limité d'offres.

Cette faible densité rend la tâche de **link prediction pertinente et non triviale**.

---

#### 3️⃣ Composantes connexes

- Nombre de composantes = **1**
- Taille de la plus grande composante = **390**

Le graphe est donc **entièrement connecté**.

Cela signifie que :

- Il existe toujours un chemin reliant n’importe quel CV à n’importe quel Job.
- Le marché simulé est globalement interconnecté.
- Il n’existe pas de sous-marchés totalement isolés.

Dans un dataset réel, on pourrait observer plusieurs composantes
correspondant à des niches professionnelles isolées.

---

#### 4️⃣ Analyse des degrés

##### Côté CV :
- Degré moyen = **4**
- Degré maximal = **4**

Chaque CV est connecté à environ 4 offres en moyenne,
et le maximum est également 4.

Cela suggère que :

- Le dataset a été généré avec un nombre fixe (ou quasi fixe) de correspondances par CV.
- Il n’y a pas de CV extrêmement polyvalents (pas de "super hubs").

---

##### Côté Job :
- Degré moyen ≈ **13.33**
- Degré maximal = **29**

Certaines offres attirent beaucoup plus de CV que d’autres.

Cela reflète :

- Des postes génériques ou populaires (forte attractivité)
- Des domaines très demandés

La forte variance côté Job indique une structure plus hétérogène que côté CV.

---

#### 5️⃣ Asymétrie structurelle

On observe une asymétrie :

- Les CV ont un degré borné (≈ 4)
- Les Jobs ont un degré très variable (jusqu'à 29)

Cela correspond à un mécanisme réaliste :

- Un candidat postule à peu d'offres
- Une offre peut recevoir beaucoup de candidatures

Cette asymétrie est typique des graphes bipartis de recommandation.

---

### Conclusion structurelle

Le graphe présente :

- Une structure clairesemée
- Une composante unique dominante
- Une forte asymétrie CV–Job

Ce contexte est particulièrement adapté pour :
- La détection de communautés
- La prédiction de liens
- La classification des profils selon leur position structurelle

## 2) Communautés détectées (Louvain)

Les communautés détectées sur la projection CV–CV représentent
des groupes de CV partageant des offres similaires.

Cela peut correspondre à :
- Domaines professionnels
- Niveaux d’expérience
- Spécialisations techniques

In [3]:
cv_comm = pd.read_csv("results/cv_with_communities.csv")

community_sizes = cv_comm["community_id"].value_counts().sort_values(ascending=False)
community_sizes.head(10)

,count
community_id,
3,60
6,50
5,49
0,46
1,37
2,36
4,22


### Interprétation des communautés détectées (projection CV–CV)

La détection Louvain a identifié **7 communautés distinctes**
avec les tailles suivantes :

- Community 3 → 60 CV
- Community 6 → 50 CV
- Community 5 → 49 CV
- Community 0 → 46 CV
- Community 1 → 37 CV
- Community 2 → 36 CV
- Community 4 → 22 CV

---

### 1️⃣ Structure globale

On observe une distribution relativement équilibrée des tailles,
avec plusieurs communautés majeures (entre 45 et 60 CV).

Cela suggère que :

- Le marché simulé est structuré en **plusieurs pôles professionnels comparables**
- Il n’existe pas une communauté ultra-dominante
- Le réseau n’est pas fragmenté en micro-niches isolées

---

### 2️⃣ Interprétation structurelle

Dans la projection CV–CV :

Deux CV sont connectés s’ils partagent des offres similaires.

Donc une communauté correspond à :

→ Un groupe de CV ayant des correspondances structurellement proches  
→ Probablement un **domaine professionnel commun**  
→ Ou un **niveau d'expérience similaire**

Les grandes communautés (3, 6, 5, 0) peuvent correspondre à :

- Data / Software
- Finance / Business
- Marketing
- Management

Les plus petites communautés (ex: community 4 → 22 CV)
peuvent représenter :

- Des profils spécialisés
- Des niches techniques
- Des combinaisons rares de compétences

---

### 3️⃣ Signification en termes de marché

La présence de plusieurs communautés significatives indique :

- Une segmentation naturelle du marché du travail
- Une cohérence structurelle entre profils et offres
- Une organisation modulaire du graphe

Cette modularité est un signal fort que :

→ Le graphe capture bien la similarité professionnelle  
→ Les communautés peuvent servir de **feature pertinente pour la classification**

---



### Conclusion

La structure communautaire du graphe confirme
l'existence de groupes professionnels distincts.

Cela valide l'utilisation de la détection de communautés
comme outil d'analyse structurelle et comme feature pour les tâches supervisées.

## 3) Link Prediction : Structure vs Sémantique

Nous comparons :
- Méthode structurelle (2-hop CN)
- Méthode sémantique (embeddings + cosine)

Critère d’évaluation : Precision@K

In [4]:
with open("results/linkpred_metrics.json", "r") as f:
    link_metrics = json.load(f)

link_metrics

{'precision_at_200_struct_cn': 0.015,
 'precision_at_500_struct_cn': 0.014,
 'precision_at_200_semantic': 0.05,
 'precision_at_500_semantic': 0.076,
 'n_candidates_scored': 20000,
 'test_edges': 240}

### Analyse comparative : Structure vs Sémantique

Les résultats obtenus sont :

| Méthode        | Precision@200 | Precision@500 |
|---------------|---------------|---------------|
| Structure (2-hop CN) | 0.015 | 0.014 |
| Sémantique (Embeddings) | 0.050 | 0.076 |

---

### 1️⃣ Performance structurelle (2-hop CN)

La méthode purement structurelle obtient une précision faible (~1.5%).

Cela signifie que :

- La topologie seule capture peu d'information prédictive.
- Le nombre de chemins 2-hop n'est pas suffisant pour discriminer correctement les correspondances.
- Le graphe étant relativement clairsemé, la structure locale est limitée.

La méthode structurelle repose uniquement sur la proximité dans le graphe,
sans tenir compte du contenu des profils ou des offres.

---

### 2️⃣ Performance sémantique (embeddings + cosine)

La méthode sémantique obtient :

- Precision@200 ≈ 5%
- Precision@500 ≈ 7.6%

Cela représente une amélioration significative par rapport à la méthode structurelle.

Interprétation :

- Les embeddings capturent efficacement la similarité textuelle.
- La correspondance CV ↔ Job est fortement dépendante du contenu sémantique.
- Le texte contient une information discriminante que la topologie seule ne reflète pas.

On observe même une augmentation de la précision en passant de 200 à 500,
ce qui suggère que le classement sémantique est relativement stable.

---

### 3️⃣ Comparaison directe

La méthode sémantique est environ :

- 3 à 5 fois plus performante que la méthode structurelle.

Cela indique que :

→ Dans ce dataset, le contenu textuel est plus informatif que la structure du graphe.
→ La similarité sémantique est un signal dominant pour le matching.

---

### 4️⃣ Implications pour un système réel

Pour un moteur de recommandation CV–Job :

- Une approche uniquement structurelle serait insuffisante.
- Une approche uniquement sémantique est déjà performante.
- La meilleure solution serait une approche hybride :
    - Score_final = α * score_structure + β * score_sémantique

Cette combinaison permettrait de capturer :
- La cohérence structurelle
- La similarité métier réelle

---

### Conclusion

Les résultats confirment que :

La modélisation textuelle via LLM (embeddings) apporte
une amélioration substantielle dans la prédiction de liens,
et constitue un élément central pour un système de matching intelligent.

## 4) Classification des CV

Nous avons testé différentes configurations :

1. Degree only
2. Degree + Community
3. Degree + Community + Embeddings

In [5]:
ablation = pd.read_csv("results/cv_ablation_results.csv")
ablation

,setting,accuracy,f1_weighted
0,degree_only,0.483333,0.314981
1,degree+community,0.483333,0.314981
2,degree+community+emb,0.983333,0.983364


### Analyse détaillée des résultats de classification

Les performances observées sont :

| Configuration | Accuracy | F1-weighted |
|---------------|----------|-------------|
| Degree only | 0.483 | 0.315 |
| Degree + Community | 0.483 | 0.315 |
| Degree + Community + Embeddings | 0.983 | 0.983 |

---

### 1️⃣ Structure seule (degree only)

La performance est faible (~48% accuracy, F1 ≈ 0.31).

Cela signifie que :

- Le degré biparti contient peu d'information discriminante.
- Tous les CV ont un degré similaire (≈4), donc la variance est faible.
- La structure locale ne suffit pas à distinguer les niveaux (junior/intermediate/senior).

Conclusion :
La topologie seule n'est pas un signal suffisant pour la classification.

---

### 2️⃣ Ajout des communautés

L’ajout de `community_id` ne change pas les performances.

Interprétation :

- Les communautés reflètent principalement des similarités métier (domaines).
- La variable cible (`level`) est probablement indépendante des clusters structurels.
- Les communautés capturent une segmentation horizontale (domaines),
  tandis que le niveau est une segmentation verticale (expérience).

Cela explique l’absence d’amélioration.

---

### 3️⃣ Ajout des embeddings (texte)

L’intégration des embeddings entraîne une performance quasi parfaite (~98%).

Cela indique que :

- Le texte du CV contient une information très discriminante.
- Les embeddings capturent efficacement les différences entre niveaux.
- Le modèle apprend des signaux sémantiques forts liés à l'expérience.

Cette amélioration spectaculaire confirme que :

→ La dimension sémantique domine la dimension structurelle dans cette tâche.

---

### 4️⃣ Analyse critique

Une performance proche de 1.0 peut indiquer :

- Une séparation très claire entre classes dans le dataset synthétique.
- Une forte corrélation entre texte et niveau (dans la génération des données).
- Un risque potentiel d’overfitting si le dataset est trop simple.

Dans un dataset réel, on s’attendrait à des performances plus modérées.

---

### Conclusion classification

Les résultats montrent clairement que :

- La structure du graphe seule est insuffisante pour prédire le niveau.
- Les communautés sont utiles pour segmentation métier, mais pas pour le niveau.
- Les embeddings textuels sont le signal dominant pour la classification.

Cela confirme que :

👉 Pour des tâches liées au profil individuel (niveau, spécialisation),
le contenu sémantique est plus informatif que la topologie globale.